In [4]:
# Project idea -> add workout counter in look studio google
# Packages required for this project:
import requests
import pandas as pd
import numpy as np
import pygsheets
import datetime
from datetime import date
import json
import webbrowser
from time import sleep
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import sys
sys.path.append(r'C:\Users\Usuario\Desktop\Learning-Testing\PyStrava\Scripts')
from Functions import *

In [6]:
ts = pd.json_normalize(r'C:\Users\Usuario\Desktop\Learning-Testing\PyStrava\Outputs\test_all_workouts.json')

NotImplementedError: 

In [20]:
all_workouts_df[['start_lat','start_long','end_lat','end_long']] = all_workouts_df[['start_lat','start_long','end_lat','end_long']].replace('no value', np.nan)

In [21]:
all_workouts_df.head(1)

,activity_id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_lat,start_long,...,average_heartrate_score,max_heartrate_score,avg_lap_time_score,lap_count_score,avg_speed_score,max_speed_score,pace_score,effort_score,effort_score_label,effort_score_rank
0,10697198078,Lunch Pickleball,2024-02-04,Pickleball,1.11,24.12,207.0,0.0,NaN,NaN,...,5,10,20,5,5,15,0,102,Low Effort,2


In [22]:
# Uploading to Google Sheet
sheet_name = 'All_Workouts_Table'
WriteToGsheet(service_file_path,spreadsheet_id,sheet_name,all_workouts_df)